<a href="https://colab.research.google.com/github/TongZhao-1106/Synthesis_NER/blob/main/A_demo_of_Fine_Tune_Llama_2_for_extracting_synthesis_info.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#A demo of fine-tuned Llama2 model for extracting:
#1) product
#2) precursors
#3) method
#from synthesis procedures




In [11]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#How to fine tune Llama 2

#Step 1: Installed All the Required Packages

In [12]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7 openai


#Step 2: Import All the Required Libraries

In [13]:
import os
#import openai
import random
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer
from transformers import pipeline


#Step 4. Generate the Dataset

In [14]:
import pandas as pd

data=pd.read_csv('/content/drive/MyDrive/LLM_dataset/train.csv')
#the path where your .csv data is

data.head()


,text,method,precourses,product
0,"Lithium nitride (99.6%, Alfa Aesar) and phosph...",ball mill,"lithium nitride, phosphorus, sulfide",Li3N·P2S5
1,Lithium meta-phosphate LiPO3 was obtained from...,ball mill,lithium phosphate,LiPO3
2,"LiI (Alfa Aeser, anhydrous, 99.95% metals basi...",ball mill,"LiI , Li2S, P2S5",35LiI-65(3Li2S-1P2S5)
3,The starting materials for preparing Li4SnS4 S...,ball mill,"Li2S, SnS2",Li4SnS4
4,"To prepare pure Li4Ti5O12, first, TiOSO4·xH2O ...",ball mill,"TiOSO4·xH2O, LiOH·H2O",Li4Ti5O12


In [15]:
# Split the data into train and test sets, with 90% in the train set
train_df = data.sample(frac=0.9, random_state=42)
test_df = data.drop(train_df.index)

# Save the dataframes to .jsonl files
train_df.to_json('/content/drive/MyDrive/LLM_dataset/train.jsonl', orient='records', lines=True)
test_df.to_json('/content/drive/MyDrive/LLM_dataset/test.jsonl', orient='records', lines=True)

#Step 5. Load the Llama 2 Model and set training parameters

In [16]:
# The model that you want to train from the Hugging Face hub
model_name = "meta-llama/Llama-2-7b-hf"

# The instruction dataset to use
dataset_name = "/content/drive/MyDrive/LLM_dataset/train.jsonl"

# Fine-tuned model name
new_model = "llama-2-7b-synthesis_ner"

################################################################################
# QLoRA parameters
################################################################################

# LoRA attention dimension
lora_r = 64

# Alpha parameter for LoRA scaling
lora_alpha = 16

# Dropout probability for LoRA layers
lora_dropout = 0.15

################################################################################
# bitsandbytes parameters
################################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

################################################################################
# TrainingArguments parameters
################################################################################

# Output directory where the model predictions and checkpoints will be stored
output_dir = "./results"

# Number of training epochs
num_train_epochs = 300

# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = False
bf16 = False

# Batch size per GPU for training
per_device_train_batch_size = 4

# Batch size per GPU for evaluation
per_device_eval_batch_size = 4

# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 1

# Enable gradient checkpointing
gradient_checkpointing = True

# Maximum gradient normal (gradient clipping)
max_grad_norm = 0.3

# Initial learning rate (AdamW optimizer)
learning_rate = 2e-4

# Weight decay to apply to all layers except bias/LayerNorm weights
weight_decay = 0.001

# Optimizer to use
optim = "paged_adamw_32bit"

# Learning rate schedule
lr_scheduler_type = "cosine"

# Number of training steps (overrides num_train_epochs)
max_steps = -1

# Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio = 0.03

# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = True

# Save checkpoint every X updates steps
save_steps = 0

# Log every X updates steps
logging_steps = 25

################################################################################
# SFT parameters
################################################################################

# Maximum sequence length to use
max_seq_length = 600

# Pack multiple short examples in the same input sequence to increase efficiency
packing = False

# Load the entire model on the GPU 0
device_map = {"": 0}

In [17]:
!huggingface-cli login
#you have to get permission from meta AI and hugging face!


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) y
Token is valid (permission: read).
Cannot authenticate through git-cr

#Step 6. Load the Datasets with the required format

In [18]:
# Load datasets
train_dataset = load_dataset('json', data_files='/content/drive/MyDrive/LLM_dataset/train.jsonl', split="train")
valid_dataset = load_dataset('json', data_files='/content/drive/MyDrive/LLM_dataset/test.jsonl', split="train")

# Preprocess datasets with a required format {'text':'The synthesis procedure'}
def formatting_func(example):
    text={}

    text['text']=f"###Synthesis procedure is: {example['text']}\n 1)The product is {example['product']} \n 2)The precursors are: {example['precourses']} \n 3)The method is: {example['method']}\n###That is all \n"

    return text
train_dataset_mapped = train_dataset.map(formatting_func,remove_columns=['method','precourses','product'])
valid_dataset_mapped = valid_dataset.map(formatting_func)
print(train_dataset_mapped[0])

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/72 [00:00<?, ? examples/s]

Map:   0%|          | 0/8 [00:00<?, ? examples/s]

{'text': '###Synthesis procedure is: a stoichiometric mixture (2:1 m) of li2s and li3ps4 was dissolved in a small quantity of anhydrous ethanol (25 ml) in an argon atmosphere. next, the mixture was heated to 90 °c under vacuum to evaporate the solvent yielding a white precipitate (not longer than 1 h) and then treated at 200 °c for 1 h to obtain the final product (li7ps6).\n 1)The product is li7ps6 \n 2)The precursors are: li2s, li3ps4 \n 3)The method is: liquid process\n###That is all \n'}


#Step 7. Train (fine tune llama2)

In [19]:
# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

# Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache = False
model.config.pretraining_tp = 1

# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training

# Load LoRA configuration
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)

# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard"
)

# Set supervised fine-tuning parameters





Your GPU supports bfloat16: accelerate training with bf16=True


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset_mapped,
    eval_dataset=valid_dataset_mapped,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)

# Train model
trainer.train()
# Save trained model
trainer.model.save_pretrained(new_model)

/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(


Map:   0%|          | 0/72 [00:00<?, ? examples/s]

Map:   0%|          | 0/8 [00:00<?, ? examples/s]

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
25,1.910100
50,1.794800
75,1.463900
100,1.229500
125,1.088100
150,0.920800
175,0.767400
200,0.605300
225,0.457600
250,0.351000


#Step 8. Test the fine-tuned model

In [ ]:
logging.set_verbosity(logging.CRITICAL)
test_text= 'Polycrystalline phosphors with compositions of Ca8MgSc(PO4)7:Dy3+ (CMSP:Dy) and Sr8MgSc(PO4)7:Dy3+ (SMSP:Dy) described in this work were prepared with a high-temperature solid-state reaction. Briefly, the constituent raw materials SrCO3 (A.R., 99.9%), CaCO3 (A.R., 99.9%), MgO (A.R., 99%), Sc2O3 (A.R., 99.99%), NH4H2PO4 (A.R., 99%), and Dy2O3 (A.R., 99.99%) were weighed according to the stoichiometric ratio. Individual batches of 10 g were weighed according to the designed stoichiometry and mixed homogeneously with the same mass of absolute ethyl alcohol as the dispersant. After a planetary ball-milling process, the obtained homogeneous slurry was placed in a Petri dish and dried in an oven. Then, the dried mixtures were put into a crucible with a lid and heated in a tubular furnace at 1250 °C for 6 h under a reducing atmosphere of 5% H2/95% N2. When cooled down to room temperature, the prepared phosphors were crushed and ground for subsequent measurements.'
prompt = '###Synthesis procedure is: '+ test_text +'\n ###The product is:'
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=500)
result = pipe(prompt)
print(result[0]['generated_text'])